In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
from src import rag, elastic_search_engine

In [5]:
elastic_keyword_searcher = elastic_search_engine.ElasticKeywordSearcher(index_name='vague_actual_keyword')
prompt_template = """
    Your are a translator from vague boss language into an everyday language. Translate
    the VAGUE statement or question based on the CONTEXT. Provide a clear and concise answer.
    VAGUE: {vague}
    
    CONTEXT: {context}
    """.strip()
    
chatgpt_4o_mini = rag.ChatGPTRAG(elastic_keyword_searcher,
                 prompt_template=prompt_template, 
                 llm_model='gpt-4o-mini')

In [8]:
chatgpt_4o_mini.rag_results('We need to become Speedy Gonzales and kick it into hyperdrive or we are going to be stuck in traffic.')

'VAGUE: We need to become Speedy Gonzales and kick it into hyperdrive or we are going to be stuck in traffic.\n\nACTUAL: We need to accelerate our development process to meet project deadlines.'